In [1]:
import asyncio
import websockets
import json

from neuron import h
from neuron.units import ms, mV

%run ../../NoteBooks/HocParse.ipynb
# This sets example_data

h.load_file('C:/Users/Research/Neuron-Web-Interface/Data/DeitcherEtAl2017/Morphs_and_trees/1148.hoc')
h.load_file('stdrun.hoc')

Section soma has 21 points!
Section axon has 172 points!
Section dend has 17 points!
	 132 more sections...


1.0

In [2]:
class WebSocketServer:
    def __init__(self, port):
        self.port = port

    async def run(self):
        async with websockets.serve(self.handler, "", self.port):
            await asyncio.Future()

    async def handler(self, websocket, path):
        while True:
            try:
                data = await websocket.recv()
                await self.handle_message(websocket, data)
            except websockets.exceptions.ConnectionClosedOK:
                continue
            except Exception as e:
                print(f"Error: {e}")

    async def handle_message(self, websocket, data):
        try:
            data = json.loads(data)
        except json.decoder.JSONDecodeError:
            print("Invalid message:", data)
            return

        request_type = data.get("message")
        if request_type == "RENDER":
            if "data" in data:
                await self.render(websocket, data["data"])
        else:
            print(f"Unknown request type: {request_type}")

    async def render(self, websocket, data):
        # Replace this with your actual render logic
        output_data = await self.run_simulaton(data)
        message = {"message": "RENDER_DATA", "data": output_data}
        print("Sending", message)
        await websocket.send(json.dumps(message))

    async def run_simulaton(self, data):
        section_lookup = {sec.name(): sec for sec in h.allsec()}
        output_data = []
        print(len(section_lookup), "sections found...")
        for section_name in section_lookup:
            section = section_lookup[section_name]
            section.Ra = data["Biophysics"]["Ra"] # Axial resistance in Ohm * cm
            section.cm = data["Biophysics"]["cm"] # Membrane capacitance in micro Farads / cm^2

            if ("soma" == section_name):
                has_hh = section.has_membrane("hh")         
                if (has_hh == False):
                    section.insert("hh")
                
                for seg in section:
                    seg.hh.gnabar = data["Biophysics"]["gnabar"] # Sodium conductance in S/cm2
                    seg.hh.gkbar = data["Biophysics"]["gkbar"] # Potassium conductance in S/cm2
                    seg.hh.gl = data["Biophysics"]["gl"] # Leak conductance in S/cm2
                    seg.hh.el = data["Biophysics"]["el"] # Reversal potential in mV

            if ("dend" in section_name or "apic" in section_name):
                has_pas = section.has_membrane("pas")
                if (has_pas == False):
                    section.insert("pas")
                
                for seg in section:
                    seg.pas.g = data["Biophysics"]["g"] # Passive conductance in S/cm2
                    seg.pas.e = data["Biophysics"]["e"] # Leak reversal potential mV 

        print(len(data["Simulation"]["Stimulation"]["Targets"]), "stim targets...")
        for stim_target in data["Simulation"]["Stimulation"]["Targets"]:
            target_seg = section_lookup[stim_target["Section"]](stim_target["Location"])
            stim = h.IClamp(target_seg)
            stim.delay = stim_target["Delay"]
            stim.dur = stim_target["Duration"]
            stim.amp = stim_target["Amplitude"]

        print(len(data["Simulation"]["Recording"]["Targets"]), "recording targets...")
        for record_target in data["Simulation"]["Recording"]["Targets"]:
            target_seg = section_lookup[record_target["Section"]](record_target["Location"])
            v = h.Vector().record(target_seg._ref_v)
            t = h.Vector().record(h._ref_t)
            
            output_data.append({ "target": record_target["Section"], "location": record_target["Location"], "voltage": v, "time": t})

        init_voltage = data["Simulation"]["Recording"]["InitialVoltage"]
        sim_span = data["Simulation"]["Recording"]["Span"]
        h.finitialize(init_voltage * mV)
        h.continuerun(sim_span * ms)

        output_data = [{ 
            "target": recording["target"], 
            "location": recording["location"], 
            "voltage": recording["voltage"].to_python(), 
            "time": recording["time"].to_python()} 
            for recording in output_data]

        return output_data

async def main():
    server = WebSocketServer(8001)
    print("Starting...")
    await server.run()

await main()


Starting...
